# Migration of results from v1 to v2

Use this notebook to migrate exiting trial results from format used by KAI DC Builder version 1.0 to the version 2.0.

## Quick start

If you followed the standard storage migration guide, this script should work without any changes. Just run all the cells.

## Prerequisites

1. Start a DSE server version 2.0 with a mounted storage volume you want to migrate the data to. Copy existing v1 (source) data to the target storage location under the `v1` subfolder. For example, if the target storage location is `$HOME/dse-storage-v2/kaidcb`, copy the v1 data to `$HOME/dse-storage-v2/kaidcb/v1`.
2. The DSE container ports 50001 and 49999 must be accessible from the Jupyter server running this notebook
3. Install DSE client libraries in the Python kernel used by the notebook

It is recommended to use `compose.nb.yml` Docker Compose file to satisfy the prerequisites 2 and 3.

## Script configuration

Use the code section below to provide the addresses and port numbers of the DSE Controller and Storage Service to the rest of the notebook. If you're using `compose.nb.yml`, the default values should work without changes.

The script will call storage migration API on the DSE server. It will make the DSE server to take each result from the v1.0 storage location, convert them to the new format, and save them in the location provided to the storage service via `--fs_url` parameter.

In [ ]:
# Must be set to the IP address or hostname where the DSE Server is running
DSE_CONTROLLER_ADDRESS = "dse"
# Must be set to the port of DSE Server specified by the startup parameter --grpc_port, default 50001
DSE_CONTROLLER_PORT = 50001
# Must be set to the IP address where the DSE Storage Service is running, usually the same as the DSE Contoller
STORAGE_SERVICE_ADDRESS="dse"
# Must be set to the the port specified by the startup parameter --storage_server, default 49999
STORAGE_SERVICE_PORT = 49999

# The results source is an fs_url pointing to the storage v1 location.
# The original files in this location will not be changed.
MIGRATION_SOURCE="osfs:///var/lib/dse/v1"

Run the cell below to initiate the migration process.

In [ ]:
import asyncio
from keysight_dse_client import Client
from keysight_dse_models.dse_models import StorageV2_MigrateStorageResponse

async def migrate_storage(client: Client):
    results = client.results

    print(f"calling MigrateStorage on fs_url {MIGRATION_SOURCE}")
    resp: StorageV2_MigrateStorageResponse = await results.migrate_storage(fs_url=MIGRATION_SOURCE)
    print(f"new result id's: {resp.import_report_ids}")
    for error in resp.errors:
        print(error)

async def main():
    client = Client(
        server_address=  DSE_CONTROLLER_ADDRESS,
        server_port=     DSE_CONTROLLER_PORT,
        storage_address= STORAGE_SERVICE_ADDRESS,
        storage_port=    STORAGE_SERVICE_PORT,
    )
    await migrate_storage(client)

await main()